All quotes below are excerpted from https://toadstyle.org/cryptopals/60.txt

```
60. Single-Coordinate Ladders and Insecure Twists

All our hard work is about to pay some dividends. Here's a list of
cool-kids jargon you'll be able to deploy after completing this
challenge:

* Montgomery curve
* single-coordinate ladder
* isomorphism
* birational equivalence
* quadratic twist
* trace of Frobenius

Not that you'll understand it all; you won't. But you'll at least be
able to silence crypto-dilettantes on Twitter.

Now, to the task at hand. In the last problem, we implemented ECDH
using a short Weierstrass curve form, like this:

    y^2 = x^3 + a*x + b

For a long time, this has been the most popular curve form. The NIST
P-curves standardized in the 90s look like this. It's what you'll see
first in most elliptic curve tutorials (including this one).

We can do a lot better. Meet the Montgomery curve:

    B*v^2 = u^3 + A*u^2 + u

Although it's almost as old as the Weierstrass form, it's been buried
in the literature until somewhat recently. The Montgomery curve has a
killer feature in the form of a simple and efficient algorithm to
compute scalar multiplication: the Montgomery ladder.

Here's the ladder:

    function ladder(u, k):
        u2, w2 := (1, 0)
        u3, w3 := (u, 1)
        for i in reverse(range(bitlen(p))):
            b := 1 & (k >> i)
            u2, u3 := cswap(u2, u3, b)
            w2, w3 := cswap(w2, w3, b)
            u3, w3 := ((u2*u3 - w2*w3)^2,
                       u * (u2*w3 - w2*u3)^2)
            u2, w2 := ((u2^2 - w2^2)^2,
                       4*u2*w2 * (u2^2 + A*u2*w2 + w2^2))
            u2, u3 := cswap(u2, u3, b)
            w2, w3 := cswap(w2, w3, b)
        return u2 * w2^(p-2)

You are not expected to understand this.

Go ahead and implement the ladder. Remember that all computations are in GF(233970423115425145524320034830162017933).
```

In [1]:
from dataclasses import dataclass
from itertools import combinations
from datetime import datetime
from random import randrange
from math import log, ceil
from pprint import pprint

from challenge_31 import do_sha256, hmac
from challenge_39 import invmod, InvModException
from challenge_57 import primegen, int_to_bytes, mini_crt, crt
from challenge_59 import tonelli_shanks, NoQuadraticResidueError

In [2]:
# I just found out that as of Python 3.8 we have a builtin modular inverse!
# instead of invmod(n, p) we can just do pow(n, -1, p)
# this builtin offers much better performance than our handrolled invmod()
# check it out:
from sys import version
assert version >= '3.8'
%timeit invmod(47056997434983591416443657936535857619, 233970423115425145524320034830162017933)
%timeit pow(47056997434983591416443657936535857619, -1, 233970423115425145524320034830162017933)

# we'll use pow() below - but let's make sure its behavior matches invmod()'s first
def test_invmod():
    p = 233970423115425145524320034830162017933
    for _ in range(10000):
        # get some n such that n has an inverse mod p
        while True:
            n = randrange(0, p)
            try:
                n_inv = invmod(n, p)
            except InvModException:
                continue
            break
        assert n_inv == pow(n, -1, p)

test_invmod()
print("Looks good!")

21.5 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
7.26 µs ± 272 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Looks good!


In [118]:
def cswap(a, b, i):
    # b,a if i else a,b
    return (b, a) if i else (a, b)  # absurdly, this is faster than the arithmetic implementation in python
    #return (b*i + a*(1-i), a*i + b*(1-i))


@dataclass
class MontyCurve:
    # reference document for formulas:
    # https://www.hyperelliptic.org/EFD/g1p/auto-montgom.html
    
    a: int
    b: int
    p: int
    
    def __post_init__(self):
        blp = int(ceil(log(self.p, 2)))  # bitlength of p
        self._i_range = range(blp)[::-1]  # used by mul
        
    def add(self, u1, u2):
        # (note: only returns u3)
        # special cases: check for zeroes, equality, inverses
        if u1 == 0: return u2
        if u2 == 0: return u1
        if u1 == u2: return self.double(u1)
        p = self.p
        if u1 == pow(u2, -1, p): return 0
        
        a, b = self.a, self.b
        
        v1 = self.get_v(u1)
        v2 = self.get_v(u2)
        
        u3 = (b*pow(v2-v1, 2, p)*pow(u2-u1, -2, p) - a - u1 - u2) % p
        return u3
    
    def add_pts(self, uv1, uv2):
        u1, v1 = uv1
        u2, v2 = uv2
        if u1 == 0: return uv2
        if u2 == 0: return uv1
        if u1 == u2:
            u3 = self.double(u1)
            v3 = self.get_v(u3)
            return (u3, v3)

        p = self.p
        if u1 == pow(u2, -1, p): return (0, 1)
        a, b = self.a, self.b
        
        du = (u2-u1) % p
        dv = (v2-v1) % p
        dui = pow(du, -1, p)  # lol dui

        u3 = (b * dv**2 * pow(dui, 2, p) - a - u1 - u2) % p
        v3 = ((2*u1 + u2 + a) * dv * dui
              - b*dv**3 * pow(dui, 3, p)
              - v1) % p

        return (u3, v3)

    def double(self, u, v=None):
        # special case: check for zero
        if u == 0: return 0
        
        a, b, p = self.a, self.b, self.p
        v = v or self.get_v(u)
        u2 = (b*pow(3*pow(u, 2, p) + 2*a*u+1, 2, p) * pow(2*b*v, -2, p)-a-u-u) % p
        return u2

    def get_v(self, u):
        a, b, p = self.a, self.b, self.p
        assert b == 1  # make sure we can get away with omitting modinv(b) in here
        rhs = (pow(u, 3, p) + a*pow(u, 2, p) + u) % p
        v_sq = (rhs * pow(b, -1, p)) % p
        v1, v2 = tonelli_shanks(rhs, p)
        return v1 if v1 < v2 else v2  # assume we want the smaller v candidate
    
    def get_pt(self, u):  # convenience wrapper on get_v
        return (u, self.get_v(u))
    
    def mul(self, u, k):
        a = self.a
        p = self.p
        u2 = 1
        w2 = 0
        u3 = u
        w3 = 1
        for i in self._i_range:
            b = 1 & (k >> i)
            u2, u3 = (u3, u2) if b else (u2, u3)
            w2, w3 = (w3, w2) if b else (w2, w3)
            u3, w3 = ((u2*u3 - w2*w3)**2 % p,
                       u * (u2*w3 - w2*u3)**2 % p)
            u2, w2 = ((u2**2 - w2**2)**2 % p,
                       4*u2*w2 * (u2**2 + a*u2*w2 + w2**2) % p)
            u2, u3 = (u3, u2) if b else (u2, u3)
            w2, w3 = (w3, w2) if b else (w2, w3)
        return (u2 * pow(w2, p-2, p)) % p


# ==== Profiling ====
# %timeit get_extra_coeffs()
# for 3 different implementations of MontyCurve.mul:
# 2.87 s ± 66.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)  cswap function (branching)
# 2.68 s ± 64.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)  cswap inlined (branching)
# 3.14 s ± 33.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)  cswap inlined (arithmetic)

---
```
Oh yeah, the curve parameters. You might be thinking that since we're
switching to a new curve format, we also need to pick out a whole new
curve. But you'd be totally wrong! It turns out that some short
Weierstrass curves can be converted into Montgomery curves.

You can perform this conversion algebraically. But it's kind of a
pain, so here you go:

    v^2 = u^3 + 534*u^2 + u

Through cunning and foresight, I have chosen this curve specifically
to have a really simple map between Weierstrass and Montgomery
forms. Here it is:

    u = x - 178
    v = y

Which makes our base point:

    (4, 85518893674295321206118380980485522083)

Or, you know. Just 4.

Anyway, implement the ladder. Verify ladder(4, n) = 0. Map some points
back and forth between your Weierstrass and Montgomery representations
and verify them.
```

In [119]:
curve = MontyCurve(a=534, b=1, p=233970423115425145524320034830162017933)
order = 233970423115425145498902418297807005944  # value copied from 59.txt

assert curve.mul(4, order) == 0

monty_base_pt = (4, curve.get_v(4))
assert monty_base_pt == (4, 85518893674295321206118380980485522083)

assert curve.mul(4, 8) == curve.add(
    curve.add(curve.add(4, 4), curve.add(4, 4)),
    curve.add(curve.add(4, 4), curve.add(4, 4))
)

assert curve.get_pt(curve.mul(4, 8)) == curve.add_pts(
    curve.add_pts(curve.add_pts(monty_base_pt, monty_base_pt),
                  curve.add_pts(monty_base_pt, monty_base_pt)),
    curve.add_pts(curve.add_pts(monty_base_pt, monty_base_pt),
                  curve.add_pts(monty_base_pt, monty_base_pt))
)

assert curve.get_pt(curve.mul(4, 9)) == curve.add_pts(curve.add_pts(
    curve.add_pts(curve.add_pts(monty_base_pt, monty_base_pt),
                  curve.add_pts(monty_base_pt, monty_base_pt)),
    curve.add_pts(curve.add_pts(monty_base_pt, monty_base_pt),
                  curve.add_pts(monty_base_pt, monty_base_pt))
), monty_base_pt)

_s = curve.mul(4, 300)
_t = curve.get_v(_s)
_u = curve.mul(4, 1337)
_v = curve.get_v(_u)
%timeit curve.mul(4, 1337)
%timeit curve.get_v(_u)
%timeit curve.add_pts((_s, _t), (_u, _v))
del _s, _t, _u, _v

AssertionError: 

In [115]:
from challenge_59 import Curve as WeierCurve
w_curve = WeierCurve(a=-95051, b=11279326, p=233970423115425145524320034830162017933)
w_base = (182, 85518893674295321206118380980485522083)

In [6]:
def to_monty(pt):
    if pt is w_curve.zero:
        return 0
    x = pt[0]
    return x - 178


def to_weier(u):
    if u == 0:
        return w_curve.zero
    p, a = curve.p, curve.a
    v1, v2 = tonelli_shanks(
        (pow(u, 3, p) + a*pow(u, 2, p) + u),
        p
    )
    return (u + 178, v1), (u + 178, v2)  # two possibilities


def test_monty_weier_conversion():
    assert to_monty(w_base) == 4
    
    for _ in range(50):
        i = randrange(0, 10000)
        w = w_curve.mul(w_base, i)
        m = curve.mul(4, i)
        
        assert to_monty(w) == m
        assert w in to_weier(m)
        
    print("All good!", flush=True)

test_monty_weier_conversion()

All good!


---
```
One nice thing about the Montgomery ladder is its lack of special
cases. Specifically, no special handling of: P1 = O; P2 = O; P1 = P2;
or P1 = -P2. Contrast that with our Weierstrass addition function and
its battalion of ifs.

And there's a security benefit, too: by ignoring the v coordinate, we
take away a lot of leeway from the attacker. Recall that the ability
to choose arbitrary (x, y) pairs let them cherry-pick points from any
curve they can think of. The single-coordinate ladder robs the
attacker of that freedom.

But hang on a tick! Give this a whirl:

    ladder(76600469441198017145391791613091732004, 11)
```

In [7]:
u = 76600469441198017145391791613091732004
curve.mul(u, 11)

0

---
```
What the heck? What's going on here?

Let's do a quick sanity check. Here's the curve equation again:

    v^2 = u^3 + 534*u^2 + u

Plug in u and take the square root to recover v.
```

In [8]:
def u_to_v(u, curve=curve):
    p = curve.p
    v_sq = (u**3 + curve.a * u**2 + u) % p
    return tonelli_shanks(v_sq, p)

try:
    u_to_v(u)
except NoQuadraticResidueError:
    print("ERROR: Square root of", u, "does not exist!")

ERROR: Square root of 76600469441198017145391791613091732004 does not exist!


---
```
You should detect that something is quite wrong. This u does not
represent a point on our curve! Not every u does.

This means that even though we can only submit one coordinate, we
still have a little bit of leeway to find invalid
points. Specifically, an input u such that u^3 + 534*u^2 + u is not a
quadratic residue can never represent a point on our curve. So where
the heck are we?

The other curve we're on is a sister curve called a "quadratic twist",
or simply "the twist". There is actually a whole family of quadratic
twists to our curve, but they're all isomorphic to each
other. Remember that that means they have the same number of points,
the same subgroups, etc. So it doesn't really matter which particular
twist we use; in fact, we don't even need to pick one.

...

If Alice chose a curve with an insecure twist, i.e. one with a
partially smooth order, then some doors open back up for Eve. She can
choose low-order points on the twisted curve, send them to Alice, and
perform the invalid-curve attack as before.

The only caveat is that she won't be able to recover the full secret
using off-curve points, only a fraction of it. But we know how to
handle that.

So:

1. Calculate the order of the twist and find its small factors. This
   one should have a bunch under 2^24.
```

In [9]:
# The ordinary curve and its twist have 2*p + 2 points between them.
# The curve's order is known, so we can take the difference to find the twist's order:
twist_order = 2*curve.p + 2 - order

print("Twist's order:", twist_order)
print("Factoring...")
%time factors = [p for p in primegen(up_to=2**24) if twist_order % p == 0 and (twist_order // p) % p != 0]
print("Small, non-repeated factors:", factors)

Twist's order: 233970423115425145549737651362517029924
Factoring...
CPU times: user 22.5 s, sys: 334 ms, total: 22.8 s
Wall time: 23.2 s
Small, non-repeated factors: [11, 107, 197, 1621, 105143, 405373, 2323367]


----
```
2. Find points with those orders. This is simple:

   a. Choose a random u mod p and verify that u^3 + A*u^2 + u is a
      nonsquare in GF(p).

   b. Call the order of the twist n. To find an element of order q,
      calculate ladder(u, n/q).
```

In [10]:
# encapsulate this search in a function to avoid cluttering the top level namespace with temp variables
def get_twist_point(fac):
    p = curve.p
    ladder = curve.mul

    while True:
        u = randrange(0, p)
        expr = (pow(u, 3, curve.p) + curve.a*pow(u, 2, p) + u) % curve.p
        try:
            tonelli_shanks(expr, p)
        except NoQuadraticResidueError: pass
        else: continue
        elem = ladder(u, twist_order // fac)
        if elem != 0:
            break

    assert twist_order % fac == 0
    assert ladder(elem, fac) == 0

    return elem

twist_points = {fac: get_twist_point(fac) for fac in factors}
pprint(twist_points)

print("\nHere's the order-11 twist subgroup we found:")
for i in range(13):
    print(curve.mul(twist_points[11], i))
print("...\n")

# note how all the i'th and 11-i'th elements are equal
# this is the cause of the 'combinatorial explosion' alluded to at the end of 60.txt
# we can probably get some "easy" wins by taking additional residues mod eg 11*107, 11*197, ...
# this would reduce our final search's work factor by a factor of 2, at the cost of some precomputation
# we'll see diminishing returns as the precomputation starts to involve larger subgroups and gets more expensive
# thus, this becomes an optimization problem - but i'm getting ahead of myself here
# we'll have to work through some more preliminaries before we get to work on solving that problem

{11: 4612483201341222105440076661179035958,
 107: 18099852405125855464264947824566507330,
 197: 225494731239453515166586115190288828970,
 1621: 115238928844861045744207102317754039233,
 105143: 229733822672992277471837182483615023120,
 405373: 103320172949680924817459278946697673994,
 2323367: 93075486678225809954488184832110610762}

Here's the order-11 twist subgroup we found:
0
4612483201341222105440076661179035958
76600469441198017145391791613091732004
105888069003703096891937904030103459645
173527332646559565669040569905840307859
1430388126279164727092494211327512206
1430388126279164727092494211327512206
173527332646559565669040569905840307859
105888069003703096891937904030103459645
76600469441198017145391791613091732004
4612483201341222105440076661179035958
0
4612483201341222105440076661179035958
...



---
```
3. Send these points to Alice to recover portions of her secret.
```

In [11]:
# implementation of Alice here is modeled on challenge 58's Bob
# cf that block's comments

def alice_coro(message):
    p, mul = curve.p, curve.mul
    #priv = randrange(0, order)
    priv = randrange(0, 100)
    pub = mul(4, priv)

    h = (yield pub)
    while True:
        secret = mul(h, priv)
        K = do_sha256(int_to_bytes(secret))
        t = hmac(K, message)
        h = (yield (message, t))

alice = alice_coro(b"no alarms and no surprises")
alice_pubkey = next(alice)
print("Alice initialized. Pubkey:", alice_pubkey)

Alice initialized. Pubkey: 37644840017132819785764168828753392334


In [12]:
def get_tag(message, guess):
    K = do_sha256(int_to_bytes(guess))
    tag = hmac(K, message)
    return tag


def recover_coefficient(g, order, message, t):
    ladder = curve.mul
    for i in range(order):
        if get_tag(message, ladder(g, i)) == t:
            return (i, order-i)
    print("coefficient not found (?!)")
    raise Exception("this should never happen")

In [13]:
coeffs = {}

print("This step's runtime varies widely. It may take a while.")
for small_order, small_pt in twist_points.items():
    print("\nRecovering possible residues mod", small_order, "...", flush=True)
    message, t = alice.send(small_pt)
    %time i1, i2 = recover_coefficient(small_pt, small_order, message, t)
    coeffs[small_order] = i1, i2

print("Done!\n")
pprint(coeffs)

This step's runtime varies widely. It may take a while.

Recovering possible residues mod 11 ...
CPU times: user 1.92 ms, sys: 0 ns, total: 1.92 ms
Wall time: 3.58 ms

Recovering possible residues mod 107 ...
CPU times: user 5.52 ms, sys: 0 ns, total: 5.52 ms
Wall time: 5.58 ms

Recovering possible residues mod 197 ...
CPU times: user 59.3 ms, sys: 0 ns, total: 59.3 ms
Wall time: 63.4 ms

Recovering possible residues mod 1621 ...
CPU times: user 94.6 ms, sys: 1.14 ms, total: 95.8 ms
Wall time: 102 ms

Recovering possible residues mod 105143 ...
CPU times: user 59 ms, sys: 0 ns, total: 59 ms
Wall time: 63 ms

Recovering possible residues mod 405373 ...
CPU times: user 70.5 ms, sys: 1.07 ms, total: 71.6 ms
Wall time: 75.6 ms

Recovering possible residues mod 2323367 ...
CPU times: user 80.2 ms, sys: 197 µs, total: 80.4 ms
Wall time: 99 ms
Done!

{11: (2, 9),
 107: (10, 97),
 197: (97, 100),
 1621: (97, 1524),
 105143: (97, 105046),
 405373: (97, 405276),
 2323367: (97, 2323270)}


In [14]:
# Let's make like a tree and generate even more residues :)


def get_extra_coeffs():
    extra_coeffs = {}
    p1 = factors[0]
    for p2 in factors[1:]:
        print(f"Recovering possible residues mod {p1}*{p2}", flush=True)
        o = p1*p2
        while True:
            pt = get_twist_point(o)
            # make sure pt has order p1*p2, not order p1 or p2
            if curve.mul(pt, p1) != 0 and curve.mul(pt, p2) != 0:
                break

        message, tag = alice.send(pt)

        candidates = [mini_crt(r1, p1, r2, p2)[0] for r2 in coeffs[p2] for r1 in coeffs[p1]]

        extra_coeffs[p1, p2] = tuple(res for res in candidates
                                     if get_tag(message, curve.mul(pt, res)) == tag)
        assert extra_coeffs[p1, p2]
        assert len(extra_coeffs[p1, p2]) == 2  # make sure we've gained information by doing this
        # NOTE: a fully automated implementation of this attack would have to omit this second assertion,
        # since there is a small chance of these residues being zero, in which case we'd end up with
        # extra_coeffs[p1, p2] == [0, 0, 0, 0]
        # this would actually be a good outcome because it'd bring us down to one possible residue
        # (rather than two)
        # but dealing with it would take some extra bookkeeping
        
    print("Done!\n")
    return extra_coeffs


%time extra_coeffs = get_extra_coeffs()
pprint(extra_coeffs)

Recovering possible residues mod 11*107
Recovering possible residues mod 11*197
Recovering possible residues mod 11*1621
Recovering possible residues mod 11*105143
Recovering possible residues mod 11*405373
Recovering possible residues mod 11*2323367
Done!

CPU times: user 66.8 ms, sys: 3.96 ms, total: 70.8 ms
Wall time: 97.8 ms
{(11, 107): (1080, 97),
 (11, 197): (97, 2070),
 (11, 1621): (97, 17734),
 (11, 105143): (97, 1156476),
 (11, 405373): (97, 4459006),
 (11, 2323367): (97, 25556940)}


In [15]:
residue_pairs = []

for t1, t2 in extra_coeffs.items():
    p1, p2 = t1
    r1, r2 = t2
    pairs = (r1%p1, r1%p2), (r2%p1, r2%p2)
    residue_pairs.append(sorted(pairs))

res_seq_1 = (coeffs[11][0],) + tuple(t[0][1] for t in residue_pairs)
res_seq_2 = (coeffs[11][1],) + tuple(t[1][1] for t in residue_pairs)

pprint(residue_pairs)
print()
print("2 possible combinations of residues (down from 128):")
print(res_seq_1)
print(res_seq_2)

[[(2, 10), (9, 97)],
 [(2, 100), (9, 97)],
 [(2, 1524), (9, 97)],
 [(2, 105046), (9, 97)],
 [(2, 405276), (9, 97)],
 [(2, 2323270), (9, 97)]]

2 possible combinations of residues (down from 128):
(2, 10, 100, 1524, 105046, 405276, 2323270)
(9, 97, 97, 97, 97, 97, 97)


---
```
4. When you've exhausted all the small subgroups in the twist, recover
   the remainder of Alice's secret with the kangaroo attack.
```

In [95]:
# commenting f here since the function is actually inlined below
#def f(u, k):  # lol fuk
#    return 1 << (u % k)  # == 2**(u % k)

vals = {}

def ec_pollard(curve, g, y, a, b, k):
    add_pts, mul, get_pt = curve.add_pts, curve.mul, curve.get_pt
    
    for key in ('xt', 'yt', 'xw', 'yw'):
        vals[key] = []
    
    pt_cache = [get_pt(mul(g, 1 << u)) for u in range(k)] # this cache lets us avoid costly calls like
                                                          # get_pt() and mul() in this func's hotspots
    N = 4 * (2**k - 1) // k  # TODO make sure this formula is correct
    N *= 100

    print(f"Starting Pollard's algorithm (k = {k}, N = {N})")
    print()
    print("Taking the tame kangaroo out for a walk.")

    xT = 0
    yT = get_pt(mul(g, b))
    for i in range(N):
        u = yT[0] % k
        f_yT = 1 << u
        xT += f_yT
        yT = add_pts(yT, pt_cache[u])
        
        vals['xt'].append(xT)
        vals['yt'].append(yT)

    print("Releasing the wild kangaroo! (this will take a while)")
    t0 = datetime.now()
    print("Wild kangaroo starting at:", t0)

    xW = 0
    yW = get_pt(y)

    bound = b - a + xT
    update_interval = 2**23 - 1
    i = 0

    while xW < bound:
        i += 1
        if i & update_interval == 0:
            progress = xW / bound
            if (progress > 0):
                ti = datetime.now() - t0
                est_duration = ti / progress
                print(f"{progress:f}% done | Appx time remaining: ({est_duration - ti} / {est_duration})")
        u = yW[0] % k
        f_yW = 1 << u
        xW += f_yW
        yW = add_pts(yW, pt_cache[u])
        
        vals['xw'].append(xW)
        vals['yw'].append(yW)
        
        if yW[0] == yT[0]:
            print("The wild kangaroo found something!")
            return b + xT - xW

In [96]:
# NOTE: this works, but it is NOT fast! this cell will take hours to evaluate

def get_candidate_dlogs():
    dlogs = []

    for residues in (res_seq_1, res_seq_2):
        residue, modulus = crt(residues, factors)
        print("Candidate residue:", residue, "mod", modulus)

        # apply transformations analogous to those in 58.txt
        g_inv = curve.mul(4, order-modulus)
        y_prime = curve.add(alice_pubkey, g_inv)
        g_prime = curve.mul(4, modulus)

        # we need to find m such that curve.mul(g_prime, m) equals y_prime
        # then our guess for the private key is just residue + m*modulus

        lb = 0
        ub = 100
        #ub = (order // modulus)  # // 2  # we only need to search the first half of the possibility space
        print(f"Searching for an index on the range [{lb}, {ub}]")
        print(f"(starting at {datetime.now()})")
        dlog = ec_pollard(curve, g_prime, y_prime, lb, ub, k=12)
        print("Candidate dlog:", dlog)
        dlogs.append(dlog)
        print()

        return dlogs

dlogs = get_candidate_dlogs()
#while (dlogs := get_candidate_dlogs()) == [None, None]:
    #pass

print("Done! Candidates:", dlogs)

Candidate residue: 37220200115549684379402940 mod 37220200115549684379403037
Searching for an index on the range [0, 100]
(starting at 2020-08-27 01:25:41.466461)
Starting Pollard's algorithm (k = 12, N = 136500)

Taking the tame kangaroo out for a walk.
Releasing the wild kangaroo! (this will take a while)
Wild kangaroo starting at: 2020-08-27 01:25:46.457602
Candidate dlog: None

Done! Candidates: [None]


In [97]:
print(len(vals['xt']))
print(vals['xt'][:15])
print([val + 97 for val in vals['xw'][:15]])
print()
print(vals['yt'][0][0])
print(curve.mul(4, 37220200115549684379403037*(98+vals['xt'][0])))
print()
print(vals['yt'][1][0])
for i in range(95, 105):
    print(curve.mul(4, 37220200115549684379403037*(i+vals['xt'][1])))


#for pt in vals['yt']:
#    if pt in vals['yw']:
#        print("o shit!!!!")
#        break

#for pt in vals['yw']:
#    if pt in vals['yt']:
#        print("whoa")
#        break



136500
[1, 33, 289, 293, 301, 302, 306, 818, 820, 1332, 1348, 1860, 2884, 3012, 3014]
[2145, 2401, 2465, 2497, 3009, 5057, 5089, 5217, 5281, 5289, 5353, 5355, 5363, 5619, 5623]

63750792713834864629903499915386397393
63750792713834864629903499915386397393

199080820642446984608044635752747605928
23608145501791020186233703885384685972
13041118279603822404611601605020549589
158988883631640302393496269327278945300
126743804564738911578487692131485054757
178090817352490716473067671841527167405
159776842731736083679237269437819421544
205761830368124404796363497722025246670
146555058712392482886585495713665463698
3047139382530231295527342473413979694
53591006328531766930463379114497154078


[199080820642446984608044635752747605928,
 199080820642446984608044635752747605928,
 199080820642446984608044635752747605928,
 199080820642446984608044635752747605928,
 199080820642446984608044635752747605928,
 199080820642446984608044635752747605928,
 199080820642446984608044635752747605928,
 199080820642446984608044635752747605928,
 199080820642446984608044635752747605928,
 199080820642446984608044635752747605928]

In [105]:
vals['yt'][2]

(220042253066906664051245951552803841246,
 112015100924642732038828388697892839851)

In [88]:
vals['xt'][:5]

[1, 33, 289, 293, 301]

In [89]:
vals['yt'][:5]

[(63750792713834864629903499915386397393,
  159109009652492911218406989594187886417),
 (199080820642446984608044635752747605928,
  46056932911115875984899410468914835401),
 (220042253066906664051245951552803841246,
  112015100924642732038828388697892839851),
 (211438720239059828636725034839139813715,
  36648333053180701422813911964255266605),
 (121027585899492782657100768530291490324,
  208219100178036415155446031928015527708)]

In [94]:
curve.mul(4, 37220200115549684379403037*(10))

68435794843181999190465757400390892864

In [121]:
curve.add(curve.mul(4, 10), curve.mul(4, 5))

126093586288113073572702262413545375317

In [123]:
curve.mul(4, 15)

126093586288113073572702262413545375317

In [124]:
p1 = curve.get_pt(curve.mul(4, 10))
p2 = curve.get_pt(curve.mul(4, 5))

In [125]:
curve.add_pts(p1, p2)

(159797793377092076215875994735688064194,
 33706392402618431063346726123231729577)

In [126]:
curve.mul(4, 5)

159797793377092076215875994735688064194

In [ ]:
# curve.add_pts is actually subtracting? wtf